In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
from shapely.validation import make_valid
from tqdm import tqdm

In [ ]:
# load flood data and reproject
#   set data input Path and flood map name
root_dir = Path ("C:/","Data","Floods","Germany", "raw_data", "SzenarioSelten") #specify the path to the folder holding the input data
print('Root: ',end='')
print(root_dir)
_flood_map_name = "DERP_RW_L.shp" #set the name for the flood map
flood_map=root_dir / _flood_map_name

#   set data output name and directory
#output_map=root_dir / "validated_geometries" / "DERP_RW_M_4326_subsample_valid_lmu.geojson"

print('File: ',end='')
print(flood_map)
assert flood_map.is_file()
gdf_to_edit=gpd.read_file(flood_map).to_crs(4326)

Root: C:\Data\Floods\Germany\raw_data\SzenarioSelten
File: C:\Data\Floods\Germany\raw_data\SzenarioSelten\DERP_RW_L.shp


In [ ]:
# make geometries valid:
gdf_to_edit.geometry = gdf_to_edit.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

In [11]:
"""
Water depth class descriptions
T_class,Description
11,Water depth - classic flood area 0–0.5 m
12,Water depth - classic flood area > 0.5–1 m
13,Water depth - classic flood area > 1–2 m
14,Water depth - classic flood area > 2–4 m
15,Water depth - classic flood area > 4 m
16,Water depth - classic flood area > 0.5–2m (only in certain areas of Saxony)
17,Water depth - classic flood area > 2 m (only in certain areas of Saxony)
21,Water depth - indicative flood area 0–0.5 m
22,Water depth - indicative flood area > 0.5–1 m
23,Water depth - indicative flood area > 1–2 m
24,Water depth - indicative flood area > 2–4 m
25,Water depth - indicative flood area > 4 m
31,Water depth - flood-protected flood area 0–0.5 m
32,Water depth - flood-protected flood area > 0.5–1 m
33,Water depth - flood-protected flood area > 1–2 m
34,Water depth - flood-protected flood area > 2–4 m
35,Water depth - flood-protected flood area > 4 m
"""
#   define dictionary to relate water depth classes to water depths
#TODO discuss way to establish depth
w_depth_dict_lower={
    '1':0.0,
    '2':0.5,
    '3':1.0,
    '4':2.0,
    '5':4.0,
    '6':0.5, #only in areas of Saxony
    '7':2.0 #only in areas of Saxony
}
w_depth_dict_mid={
    '1':0.25,
    '2':0.75,
    '3':1.5,
    '4':3.0,
    '5':4.0,
    '6':1.25, #only in areas of Saxony
    '7':2.0 #only in areas of Saxony
}
w_depth_dict_upper={
    '1':0.5,
    '2':1.0,
    '3':2.0,
    '4':4.0,
    '5':4.0,
    '6':2.0, #only in areas of Saxony
    '7':2.0 #only in areas of Saxony
}

In [17]:
wdepth_class=[]
wdepth_l=[]
wdepth_m=[]
wdepth_u=[]
farea=[]

for i in range(len(gdf_to_edit)):
    farea.append(int(str(gdf_to_edit['T_class'][i])[0]))
    wdepth_class.append(int(str(gdf_to_edit['T_class'][i])[-1]))
    wdepth_l.append(float(w_depth_dict_lower[str(gdf_to_edit['T_class'][i])[-1]]))
    wdepth_m.append(float(w_depth_dict_mid[str(gdf_to_edit['T_class'][i])[-1]]))
    wdepth_u.append(float(w_depth_dict_upper[str(gdf_to_edit['T_class'][i])[-1]]))

gdf = gdf_to_edit.assign(w_depth_l=wdepth_l,w_depth_m=wdepth_m,w_depth_u=wdepth_u, depth_class=wdepth_class, flood_area=farea)


In [18]:
output_map = Path ("C:/","Data","Floods","Germany", "interim_data", "DERP_RW_L_4326_valid_lmu.geojson")
gdf.to_file(output_map)

In [3]:
# make basin geometries valid

basins_path="C:/Data/Floods/basins/hybas_eu_lev01-12_v1c/hybas_eu_lev08_v1c.shp"
basins_path_valid="C:/Data/Floods/basins/hybas_eu_lev01-12_v1c/hybas_eu_lev08_v1c_valid.shp"
basins=gpd.read_file(basins_path)
basins.geometry = basins.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)
basins.to_file(basins_path_valid)


Intersect basins with flood map

In [2]:
import os
from qgis.core import QgsApplication

# Set the path to the QGIS installation
qgis_path = r'C:\Users\peregrin\AppData\Local\anaconda3\envs\qgis_env\Library\bin\qgis.exe'

# Set the environment variables for QGIS
os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = os.path.join(qgis_path, 'apps', 'Qt5', 'plugins', 'platforms')
os.environ['PATH'] += os.pathsep + os.path.join(qgis_path, 'apps', 'qgis', 'bin')
os.environ['PYTHONPATH'] += os.pathsep + os.path.join(qgis_path, 'apps', 'qgis', 'python')
os.environ['LD_LIBRARY_PATH'] = os.path.join(qgis_path, 'apps', 'qgis', 'lib')

# Initialize QGIS application
QgsApplication.setPrefixPath(qgis_path, True)
qgs = QgsApplication([], False)
qgs.initQgis()

# Now you can use QGIS processing tools
from qgis.analysis import QgsNativeAlgorithms
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())

import processing
# intersect basin polygons with flood map
processing.run("native:intersection", {'INPUT':'C:/Data/Floods/Germany/interim_data/DERP_RW_H_4326_valid_lmu.geojson','OVERLAY':'C:/Data/Floods/basins/hybas_eu_lev01-12_v1c/hybas_eu_lev08_v1c_valid.shp','INPUT_FIELDS':['flood_area','depth_class','w_depth_l', 'w_depth_m', 'w_depth_u'],'OVERLAY_FIELDS':['HYBAS_ID','NEXT_DOWN','NEXT_SINK','MAIN_BAS','DIST_SINK','DIST_MAIN','SUB_AREA','UP_AREA','PFAF_ID','ORDER'],'OVERLAY_FIELDS_PREFIX':'','OUTPUT':'C:/Data/Floods/Germany/basin_intersections/DERP_RW_H_4326_hybas_intersections.geojson','GRID_SIZE':None})

# exit QGIS application
qgs.exitQgis()


Save basin intersections into separate files

In [2]:
import glob
#   set data input Path and flood map name 
root_dir = Path ("C:/","Data","Floods","Germany", "basin_intersections") #specify the path to the folder holding the input data

print('Root: ',end='')
print(root_dir)

# Find all .geojson and .shp files in the root directory
file_list = glob.glob(str(root_dir / '*.geojson')) + glob.glob(str(root_dir / '*.shp'))

# Print the list of files
print(file_list)



Root: C:\Data\Floods\Germany\basin_intersections
['C:\\Data\\Floods\\Germany\\basin_intersections\\DERP_RW_H_4326_hybas_intersections.geojson', 'C:\\Data\\Floods\\Germany\\basin_intersections\\DERP_RW_L_4326_hybas_intersections.geojson', 'C:\\Data\\Floods\\Germany\\basin_intersections\\DERP_RW_M_4326_hybas_intersections.geojson']


In [3]:
def split_by_hybas_id(gdf, flood_map, output_map_dir):
    _flood_map = flood_map.split('\\')[-1].split('_hybas_')[0]
    # Iterate over unique hybas_ids
    for unique_hybas_id in tqdm(gdf['HYBAS_ID'].unique()):
        output_file=f'flood_{_flood_map}_{unique_hybas_id}.geojson'
        output_path=Path(output_map_dir) / output_file
        # Select rows with the current unique value
        subset = gdf[gdf['HYBAS_ID'] == unique_hybas_id]  
        # Export the subset as GeoJSON in the output directory
        subset.to_file(output_path, driver='GeoJSON')
    
n_map=0
len_file_list=len(file_list)
for flood_map in file_list:
    n_map+=1
    print(f'File {n_map}/{len_file_list}: ',end='')
    print(flood_map)
    assert Path(flood_map).is_file()
    output_map_dir=flood_map.split('.')[0]
    flood_map_gdf=gpd.read_file(flood_map)
    map_out=split_by_hybas_id(flood_map_gdf, flood_map, output_map_dir)
    

File 1/3: C:\Data\Floods\Germany\basin_intersections\DERP_RW_H_4326_hybas_intersections.geojson


100%|██████████| 61/61 [3:01:25<00:00, 178.45s/it]   


File 2/3: C:\Data\Floods\Germany\basin_intersections\DERP_RW_L_4326_hybas_intersections.geojson


100%|██████████| 61/61 [06:53<00:00,  6.79s/it]


File 3/3: C:\Data\Floods\Germany\basin_intersections\DERP_RW_M_4326_hybas_intersections.geojson


100%|██████████| 61/61 [06:09<00:00,  6.06s/it]


In [4]:
flood_event=gpd.read_file(r'C:\Data\Floods\Germany\basin_intersections\DERP_RW_M_4326_hybas_intersections\flood_DERP_RW_M_4326_2080410430.geojson')

In [5]:
flood_event.head()

,flood_area,depth_class,w_depth_l,w_depth_m,w_depth_u,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ORDER,geometry
0,1,1,0.0,0.25,0.5,2080410430,2080410540,2080023010,2080023010,465.9,465.9,201.7,201.8,23261088,3,"MULTIPOLYGON (((7.98655 50.73515, 7.98655 50.7..."
1,1,1,0.0,0.25,0.5,2080410430,2080410540,2080023010,2080023010,465.9,465.9,201.7,201.8,23261088,3,"MULTIPOLYGON (((7.90384 50.75885, 7.90384 50.7..."
2,1,2,0.5,0.75,1.0,2080410430,2080410540,2080023010,2080023010,465.9,465.9,201.7,201.8,23261088,3,"MULTIPOLYGON (((7.98638 50.73521, 7.98638 50.7..."
3,1,2,0.5,0.75,1.0,2080410430,2080410540,2080023010,2080023010,465.9,465.9,201.7,201.8,23261088,3,"MULTIPOLYGON (((7.88675 50.77816, 7.88673 50.7..."
4,1,3,1.0,1.50,2.0,2080410430,2080410540,2080023010,2080023010,465.9,465.9,201.7,201.8,23261088,3,"MULTIPOLYGON (((7.98329 50.73633, 7.98327 50.7..."
